In [1]:
import torch
import torch.nn as nn
import os
from covid_ode import sidartha_ode
#from ode_nn import Dataset, train_epoch, eval_epoch, get_lr
import numpy as np
import pandas as pd
import torch.nn.functional as F
from torch.utils import data
import matplotlib.pyplot as plt
import random
import warnings

from tqdm.notebook import tqdm
# from ode_nn import Dataset_graph, train_epoch_graph, eval_epoch_graph, get_lr
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
canada = pd.read_csv('canada.csv')

In [3]:
CANADA_POPULATION = 38.01e6

In [4]:
canada.shape

(555, 4)

In [5]:
canada

,Confirmed,Recovered,Deaths,Last_Update
0,4.0,0.0,0.0,2020-01-28 23:00:00
1,4.0,0.0,0.0,2020-01-29 19:30:00
2,4.0,0.0,0.0,2020-01-30 16:00:00
3,4.0,0.0,0.0,2020-01-31 23:59:00
4,4.0,0.0,0.0,2020-02-01 18:12:00
...,...,...,...,...
550,1434967.0,1404107.0,26599.0,2021-08-01 04:21:50
551,1435347.0,1404244.0,26601.0,2021-08-02 04:21:36
552,1435683.0,1404327.0,26602.0,2021-08-03 04:21:39
553,1437713.0,1405529.0,26608.0,2021-08-04 04:21:25


In [6]:
canada = canada.values[:, 0:3] / CANADA_POPULATION

In [7]:
canada[:, 0:3]

array([[1.0523546435148645e-07, 0.0, 0.0],
       [1.0523546435148645e-07, 0.0, 0.0],
       [1.0523546435148645e-07, 0.0, 0.0],
       ...,
       [0.03777119179163378, 0.036946250986582475, 0.0006998684556695607],
       [0.03782459878979216, 0.0369778742436201, 0.0007000263088660878],
       [0.0378533543804262, 0.03698950276243094, 0.0007003157063930545]],
      dtype=object)

In [8]:
##################################################################
test_idx = 131

# Learning Rate
lr = 0.01

# number of historic data points for fitting
input_steps = 10 

# forecasting horizon
output_steps = 7

# number of epochs for training
num_epochs = 20000

# select data for training
data = canada[test_idx-input_steps:test_idx+output_steps]   # only 1 training sample
y_exact = data[:,:input_steps]

##################################################################

model = sidartha_ode.AutoOdeSIDARTHE(len_data = output_steps+input_steps).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 1000, gamma=0.9)
loss_fun = torch.nn.MSELoss()
min_loss = 1

##################################################################
for e in tqdm(range(num_epochs)):
    optimizer.zero_grad()
    y_approx = model(input_steps)
    import pdb; pdb.set_trace()
    loss = loss_fun(y_approx[:,:,-3:], y_exact[:,:input_steps,-3:])
    loss.backward()
    optimizer.step()
    scheduler.step()
######## Weighted Loss ########
#     loss_weight = weight_fun(input_steps, function = "sqrt", feat_weight = True)
#     loss = torch.mean(loss_weight*loss_fun(y_approx[:,:,-3:], y_exact[:,:input_steps,-3:])) 

######## A few constraints that can potential improve the model ########
#     positive_constraint = loss_fun(F.relu(-model.beta), torch.tensor(0.0).float().to(device))
#     diagonal_constraint = loss_fun(torch.diagonal(model.A, 0),torch.tensor(1.0).float().to(device))
#     initial_constraint = loss_fun(model.init_S + model.init_E + model.init_I + model.init_R + model.init_U, torch.tensor(1.0).float().to(device))
#     loss += initial_constraint + positive_constraint + diagonal_constraint 
   
    if loss.item() < min_loss:
        best_model = model
        min_loss = loss.item()
    optimizer.zero_grad()
    loss.backward(retain_graph=True)
    optimizer.step()
#     if e%1000 == 0:
#         y_approx2 = model(data.shape[1]).data.numpy()
#         y_exact2 = data.data.numpy()
#         print(list_csv[test_idx][:10])
#         #torch.mean(torch.abs(y_approx - y_exact)[:,-7:]).data, torch.mean(torch.abs(y_approx - y_exact)[:,30:]).data
#         for i in range(3):
#             print(np.mean(np.abs(y_approx2*scaler - y_exact2*scaler)[:,-7:, i]))



  0%|          | 0/20000 [00:00<?, ?it/s]

> <ipython-input-8-7ed4544c6093>(34)<module>()
     32     y_approx = model(input_steps)
     33     import pdb; pdb.set_trace()
---> 34     loss = loss_fun(y_approx[:,:,-3:], y_exact[:,:input_steps,-3:])
     35     loss.backward()
     36     optimizer.step()



ipdb>  y_approx


tensor([[5.0000e-01, 0.0000e+00, 5.0000e-01,  ..., 5.0000e-01, 0.0000e+00,
         0.0000e+00],
        [5.0170e-01, 1.6923e-03, 4.9921e-01,  ..., 5.0009e-01, 4.2561e-04,
         5.0054e-05],
        [5.0340e-01, 3.3857e-03, 4.9842e-01,  ..., 5.0017e-01, 8.5157e-04,
         1.0012e-04],
        ...,
        [       nan,        nan,        nan,  ...,        nan,        nan,
                nan],
        [       nan,        nan,        nan,  ...,        nan,        nan,
                nan],
        [       nan,        nan,        nan,  ...,        nan,        nan,
                nan]])


ipdb>  y_approx.shape


...

tensor([0.7147, 0.1853, 0.4409, 0.3431, 0.6859, 0.5096, 0.0448, 0.0051])


ipdb>  y_approx[200]


tensor([1.1171, 0.4831, 0.4267, 0.2624, 0.8167, 0.5211, 0.0968, 0.0102])


ipdb>  q


BdbQuit: 

In [ ]:
data, y_exact